The following command attempts to fix some issues (ICU stay ID is missing) and removes the events that have missing information

In [1]:
import pandas as pd
import os

from tqdm import tqdm



In [2]:
subjects_root_path = 'data/root'


In [3]:
def is_subject_folder(x):
    # isdigit() 方法检测字符串是否只由数字组成。
    return str.isdigit(x)
subdirectories = os.listdir(subjects_root_path)
# 得到文件名称全部是数字的文件夹
subjects = list(filter(is_subject_folder, subdirectories))
# 所有的icustays的病人个数
print(len(subjects))

33798


In [4]:
s = 3
#stays_df = pd.read_csv(os.path.join(subjects_root_path, str(s), 'stays.csv'), index_col=False,
                            #dtype={'HADM_ID': str, "ICUSTAY_ID": str})
#print(stays_df)
#stays_df['ICUSTAY_ID'].isnull().any()

list1 = [None, 1, 2, None, None]
isnull_df = pd.Series(list1)
print(isnull_df)
# 判断isnull_df里面是否有空值，只要有空值就为True
print(isnull_df.isnull().any())
print(isnull_df.isnull().sum())

0    NaN
1    1.0
2    2.0
3    NaN
4    NaN
dtype: float64
True
3


In [5]:
n_events = 0                   # total number of events
empty_hadm = 0                 # HADM_ID is empty in events.csv. We exclude such events.
no_hadm_in_stay = 0            # HADM_ID does not appear in stays.csv. We exclude such events.
no_icustay = 0                 # ICUSTAY_ID is empty in events.csv. We try to fix such events.
recovered = 0                  # empty ICUSTAY_IDs are recovered according to stays.csv files (given HADM_ID)
could_not_recover = 0          # empty ICUSTAY_IDs that are not recovered. This should be zero.
icustay_missing_in_stays = 0   # ICUSTAY_ID does not appear in stays.csv. We exclude such events.

In [8]:
subject = str(3)
stays_df = pd.read_csv('data/root/3/stays.csv', index_col=False, dtype={'HADM_ID': str, "ICUSTAY_ID": str})

In [10]:
# 遍历这些病人的文件夹
#for subject in tqdm(subjects, desc='Iterating over subjects'):
'''
stays.csv的每行记录的内容是：病人每次住院的记录（每次住院只能有一个HADM-ID\icustay_id）
'''
subject = str(3)
stays_df = pd.read_csv(os.path.join(subjects_root_path, subject, 'stays.csv'), index_col=False,
                        dtype={'HADM_ID': str, "ICUSTAY_ID": str})
stays_df.columns = stays_df.columns.str.upper()
# assert that there is no row with empty ICUSTAY_ID or HADM_ID
assert(not stays_df['ICUSTAY_ID'].isnull().any())
assert(not stays_df['HADM_ID'].isnull().any())

# assert there are no repetitions of ICUSTAY_ID or HADM_ID
# since admissions with multiple ICU stays were excluded
assert(len(stays_df['ICUSTAY_ID'].unique()) == len(stays_df['ICUSTAY_ID']))
# 如果患者在住院期间发生了ICU的转移，就会出现同一个HADM_ID对应不同的ICUSTAY_ID，
# 所以HADM_ID去重之后就与去重之前的个数不再相等
assert(len(stays_df['HADM_ID'].unique()) == len(stays_df['HADM_ID']))

events_df = pd.read_csv(os.path.join(subjects_root_path, subject, 'events.csv'), index_col=False,
                        dtype={'HADM_ID': str, "ICUSTAY_ID": str})
print(events_df)
events_df.columns = events_df.columns.str.upper()
n_events += events_df.shape[0]
    # we drop all events for them HADM_ID is empty
# TODO: maybe we can recover HADM_ID by looking at ICUSTAY_ID
empty_hadm += events_df['HADM_ID'].isnull().sum()
print(events_df['HADM_ID'].isnull().sum())
events_df = events_df.dropna(subset=['HADM_ID'])

merged_df = events_df.merge(stays_df, left_on=['HADM_ID'], right_on=['HADM_ID'],
                            how='left', suffixes=['', '_r'], indicator=True)
print(merged_df)
# we drop all events for which HADM_ID is not listed in stays.csv
# since there is no way to know the targets of that stay (for example mortality)
no_hadm_in_stay += (merged_df['_merge'] == 'left_only').sum()
merged_df = merged_df[merged_df['_merge'] == 'both']

   



      SUBJECT_ID   HADM_ID ICUSTAY_ID            CHARTTIME  ITEMID VALUE  \
0              3       NaN        NaN  2101-10-12 16:07:00   50820  7.39   
1              3       NaN        NaN  2101-10-12 18:17:00   50800   ART   
2              3       NaN        NaN  2101-10-12 18:17:00   50802    -1   
3              3       NaN        NaN  2101-10-12 18:17:00   50804    22   
4              3       NaN        NaN  2101-10-12 18:17:00   50808  0.93   
...          ...       ...        ...                  ...     ...   ...   
3427       11318  178137.0   241703.0  2119-09-22 20:00:00   40054   NaN   
3428       11318  178137.0   241703.0  2119-09-22 21:00:00   40054   NaN   
3429       11318  178137.0   241703.0  2119-09-22 23:00:00   40054   NaN   
3430       11318  178137.0   241703.0  2119-09-23 11:00:00   40054   NaN   
3431       11318  178137.0   241703.0  2119-09-23 11:30:00   40054   NaN   

     VALUEUOM  
0       units  
1         NaN  
2       mEq/L  
3       mEq/L  
4      

In [ ]:
 # if ICUSTAY_ID is empty in stays.csv, we try to recover it
    # we exclude all events for which we could not recover ICUSTAY_ID
    cur_no_icustay = merged_df['ICUSTAY_ID'].isnull().sum()
    no_icustay += cur_no_icustay
    merged_df.loc[:, 'ICUSTAY_ID'] = merged_df['ICUSTAY_ID'].fillna(merged_df['ICUSTAY_ID_r'])
    recovered += cur_no_icustay - merged_df['ICUSTAY_ID'].isnull().sum()
    could_not_recover += merged_df['ICUSTAY_ID'].isnull().sum()
    merged_df = merged_df.dropna(subset=['ICUSTAY_ID'])

    # now we take a look at the case when ICUSTAY_ID is present in events.csv, but not in stays.csv
    # this mean that ICUSTAY_ID in events.csv is not the same as that of stays.csv for the same HADM_ID
    # we drop all such events
    icustay_missing_in_stays += (merged_df['ICUSTAY_ID'] != merged_df['ICUSTAY_ID_r']).sum()
    merged_df = merged_df[(merged_df['ICUSTAY_ID'] == merged_df['ICUSTAY_ID_r'])]

    to_write = merged_df[['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUEUOM']]
    to_write.to_csv(os.path.join(args.subjects_root_path, subject, 'events.csv'), index=False)

assert(could_not_recover == 0)
print('n_events: {}'.format(n_events))
print('empty_hadm: {}'.format(empty_hadm))
print('no_hadm_in_stay: {}'.format(no_hadm_in_stay))
print('no_icustay: {}'.format(no_icustay))
print('recovered: {}'.format(recovered))
print('could_not_recover: {}'.format(could_not_recover))
print('icustay_missing_in_stays: {}'.format(icustay_missing_in_stays))

In [11]:
df = pd.read_csv('MIMIC-III Clinical Database 1.4/chartevent_blocks/CHARTEVENTS_1.CSV')

In [14]:
df.dtypes

ROW_ID            int64
SUBJECT_ID        int64
HADM_ID           int64
ICUSTAY_ID      float64
ITEMID            int64
CHARTTIME        object
STORETIME        object
CGID            float64
VALUE           float64
VALUENUM        float64
VALUEUOM         object
WARNING           int64
ERROR             int64
RESULTSTATUS    float64
STOPPED         float64
dtype: object

In [17]:
for i in range(10000000000):
    print(df[df['SUBJECT_ID']==i])

Empty DataFrame
Columns: [ROW_ID, SUBJECT_ID, HADM_ID, ICUSTAY_ID, ITEMID, CHARTTIME, STORETIME, CGID, VALUE, VALUENUM, VALUEUOM, WARNING, ERROR, RESULTSTATUS, STOPPED]
Index: []
Empty DataFrame
Columns: [ROW_ID, SUBJECT_ID, HADM_ID, ICUSTAY_ID, ITEMID, CHARTTIME, STORETIME, CGID, VALUE, VALUENUM, VALUEUOM, WARNING, ERROR, RESULTSTATUS, STOPPED]
Index: []
Empty DataFrame
Columns: [ROW_ID, SUBJECT_ID, HADM_ID, ICUSTAY_ID, ITEMID, CHARTTIME, STORETIME, CGID, VALUE, VALUENUM, VALUEUOM, WARNING, ERROR, RESULTSTATUS, STOPPED]
Index: []
Empty DataFrame
Columns: [ROW_ID, SUBJECT_ID, HADM_ID, ICUSTAY_ID, ITEMID, CHARTTIME, STORETIME, CGID, VALUE, VALUENUM, VALUEUOM, WARNING, ERROR, RESULTSTATUS, STOPPED]
Index: []
Empty DataFrame
Columns: [ROW_ID, SUBJECT_ID, HADM_ID, ICUSTAY_ID, ITEMID, CHARTTIME, STORETIME, CGID, VALUE, VALUENUM, VALUEUOM, WARNING, ERROR, RESULTSTATUS, STOPPED]
Index: []
Empty DataFrame
Columns: [ROW_ID, SUBJECT_ID, HADM_ID, ICUSTAY_ID, ITEMID, CHARTTIME, STORETIME, CGID, VA

KeyboardInterrupt: 

In [26]:
mimic3_path = 'MIMIC-III Clinical Database 1.4'
outputevents = pd.read_csv(os.path.join(mimic3_path, 'OUTPUTEVENTS.csv'))
labevents = pd.read_csv(os.path.join(mimic3_path, 'LABEVENTS.csv'))

In [36]:
icustays = pd.read_csv('data/csvs/all_stays.csv', index_col = False)
subjects = icustays.SUBJECT_ID.unique()

In [42]:
if 'ICUSTAY_ID' not in labevents:
    labevents['ICUSTAY_ID'] = ''
for s in subjects:
    # 写入lab和out信息
    print(f'当前处理的病人ID是: {s}')
    mask_lab = labevents['SUBJECT_ID'] == s
    mask_out = outputevents['SUBJECT_ID'] == s
    lab = labevents[mask_lab][obs_header]
    print('*************lab 得到的信息***************')
    print(lab)
    out = outputevents[mask_out][obs_header]
    print('**************out 得到的信息***************')
    print(out)
    

当前处理的病人ID是: 268
*************lab 得到的信息***************
        SUBJECT_ID   HADM_ID ICUSTAY_ID            CHARTTIME  ITEMID  \
169707         268  110404.0             2198-02-16 10:48:00   51093   
169708         268  110404.0             2198-02-16 10:48:00   51100   
169709         268  110404.0             2198-02-16 10:48:00   51463   
169710         268  110404.0             2198-02-16 10:48:00   51464   
169711         268  110404.0             2198-02-16 10:48:00   51466   
...            ...       ...        ...                  ...     ...   
179288         268  110404.0             2198-02-16 09:36:00   50826   
179289         268  110404.0             2198-02-16 09:36:00   50827   
179290         268  110404.0             2198-02-16 09:36:00   50828   
179291         268  110404.0             2198-02-16 10:48:00   51082   
179292         268  110404.0             2198-02-16 10:48:00   51087   

                   VALUE VALUEUOM  
169707               374  mOsm/kg  
169708   

KeyboardInterrupt: 

In [33]:
obs_header = ['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUEUOM']
print(outputevents[outputevents['SUBJECT_ID']==11318][obs_header])
print(labevents[labevents['SUBJECT_ID']==3][['SUBJECT_ID', 'HADM_ID', 'CHARTTIME', 'ITEMID', 'VALUE', 'VALUEUOM']])

         SUBJECT_ID   HADM_ID  ICUSTAY_ID            CHARTTIME  ITEMID  VALUE  \
345           11318  178137.0    241703.0  2119-09-21 17:00:00   40085    NaN   
346           11318  178137.0    241703.0  2119-09-22 01:00:00   40085    0.0   
462           11318  178137.0    241703.0  2119-09-21 20:00:00   40055  800.0   
463           11318  178137.0    241703.0  2119-09-21 21:45:00   40055  120.0   
464           11318  178137.0    241703.0  2119-09-21 22:30:00   40055  200.0   
...             ...       ...         ...                  ...     ...    ...   
2799660       11318  103062.0    269413.0  2123-02-26 23:00:00   40055   80.0   
2800292       11318  111001.0    274404.0  2122-11-03 15:00:00   40055  220.0   
2806113       11318  198530.0    227112.0  2123-09-29 16:00:00   40055  100.0   
2806114       11318  198530.0    227112.0  2123-09-30 01:00:00   40055   45.0   
2814620       11318  116974.0    294101.0  2123-04-25 21:00:00   40055  100.0   

        VALUEUOM  
345     